# Debugging the code to fix the total runs for each players

SK Raina's profile from [iplt20.com]

http://cricwaves.com/ and go to Twenty20 link.

https://www.cricwaves.com/cricket/tour/113/ipl-2008/2/80/suresh-raina/PlayerStats.html
https://www.cricwaves.com/cricket/tour/44/ipl-2009/2/80/suresh-raina/PlayerStats.html

Here is the summary from [iplt20 website](https://www.iplt20.com/teams/chennai-super-kings/squad/14/suresh-raina)

```
#Batting and Fielding	
Year    Mat	No	Runs	HS	Ave	BF	SR	100	50	4s	6s	CT	ST
2019	17	1	383	59	23.93	314	121.97	0	3	45	9	7	0
2018	15	3	445	75*	37.08	336	132.44	0	4	46	12	9	0
2017	14	3	442	84	40.18	307	143.97	0	3	42	13	4	0
2016	15	1	399	75	28.50	312	127.88	0	3	39	10	7	0
2015	17	2	374	62	24.93	304	123.02	0	2	31	16	11	0
2014	16	3	523	87	40.23	358	146.08	0	5	51	19	11	0
2013	18	4	548	100	42.15	365	150.13	1	4	50	18	10	0
2012	19	1	441	73	25.94	325	135.69	0	1	36	19	11	0
2011	16	2	438	73*	31.28	325	134.76	0	4	36	17	4	0
2010	16	5	520	83*	47.27	364	142.85	0	4	45	22	10	0
2009	14	0	434	98	31.00	308	140.90	0	2	37	21	7	0
2008	16	3	421	55*	38.27	296	142.22	0	3	35	18	10	0
#Career	 193	28	5368	100*	33.34	3,914	137.14	1	38	493	194	101	0
```

years with inconsistency
- 2008: a difference of 1 ball (find that out) (My code 
- 2009: all good
- 2010: iplt20 says `520 (364)` but I got 

**2009**

https://www.cricwaves.com/cricket/tour/44/ipl-2009/2/80/suresh-raina/PlayerStats.html

Faulty one: 
    
- 2	SK Raina	41	27	Delhi Daredevils	180	151.292	392189
- the cricwaves.com shows this match his scores: 41(28) not 41(27) as I found.

- 11	SK Raina	52	36	Kolkata Knight Riders	188	144.044	392231
- cricwaves.com shows the score 52(37)




In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob, yaml, pickle
import datetime

from utils import get_match_list


data_dir='../datasets/ipl/yaml/'

In [4]:
def debug_read_match_scores(f, data_dir='./'):
    fil=os.path.join(data_dir, f )
    
    with open(fil, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

    try:
        match_date=data['info']['dates'][0].strftime('%Y-%m-%d')
    except:
        match_date=data['info']['dates'][0]
    
    season = match_date.split('-')[0]
    teams  = data['info']['teams']
    winner = data['info']['outcome'].get('winner', None)
    toss_winner = data['info']['toss'].get('winner', None)
    
    
    def add_player(player, scorecard, season=season):
        if player not in scorecard:
            scorecard[player] = {'0':0, '1':0, '2':0, '3':0, '4':0, '5':0, '6':0,
                                 'Runs':0, 'BF':0, 'NO':True, 'Team':' ', 'Against':' ',
                                 'Win':False, 'Toss':False, 'team-total':0, 'season':season}
    batting_card = {}
    
    count = 0
    for i, inn in enumerate(data['innings']):
        inn_name     = list(inn.keys())[0]

        batting_team = data['innings'][i][inn_name]['team']
        bowling_team = [team for team in  teams if team!=batting_team][0]

        batting_card_inn={}
        runs_bowler={}
        runs_extra = 0
        runs_total = 0
        wkts       = 0
        
        for delivery in data['innings'][i][inn_name]['deliveries']:
            deliv    = list(delivery.items())[0]
            
            ball     = deliv[0]
            batsman  = deliv[1]['batsman'].strip()
            bowler   = deliv[1]['bowler'].strip()
            runs_bat = deliv[1]['runs'].get('batsman', 0)
            runs_ext = deliv[1]['runs'].get('extras',  0)
            runs_tot = deliv[1]['runs'].get('total',   0)
            
            if batsman=='SK Raina':
                print (deliv[1]['runs'])
                count +=1
                if 'extras' in deliv[1]:
                    print (deliv[1])
            
            add_player(batsman, batting_card_inn)

            # counter for each runs (1, 2, 3, 4, 5, 6)
            batting_card_inn[batsman][str(runs_bat)] += 1
            
            # counter for total batsman run
            batting_card_inn[batsman]['Runs']        += runs_bat
            
            # counter for toal balls faced [ball will be removed later if it's a wide]
            batting_card_inn[batsman]['BF']          += 1
            
            if 'extras' in deliv[1]:
                if 'wides' in deliv[1]['extras']:
                    batting_card_inn[batsman]['BF']  -= 1 # remove the ball from batsman's account

            runs_extra += runs_ext
            runs_total += runs_tot
            
            if deliv[1].get('wicket', None):
                wkts += 1
                player_out = deliv[1]['wicket']['player_out']
                
                # for case when player is runout without facing a ball
                add_player(player_out, batting_card_inn)
                batting_card_inn[player_out]['NO']=False
                
            batting_card_inn[batsman]['Team']     = batting_team
            batting_card_inn[batsman]['Against']  = bowling_team
            
            if batting_team==winner:
                batting_card_inn[batsman]['Win']  = True
                
            if batting_team==toss_winner:
                batting_card_inn[batsman]['Toss'] = True
            
        for b in batting_card_inn.keys():
            batting_card_inn[b]['team-total']     = runs_total

        batting_card.update(batting_card_inn)
        
        print ('count:',count)

    df=pd.DataFrame(batting_card).T
    df.reset_index(inplace=True)
    df.rename(columns={"index": "batsman"}, inplace=True)
    df['date']=match_date
    df['match-id']=f.split('.')[0]
    df['SR'] = 100*df['Runs']/(df['BF']+0.1)
    return df

match_id = '392189'
dff=debug_read_match_scores(match_id+'.yaml', data_dir=data_dir)
dff[dff.batsman=='SK Raina']



count: 0
{'batsman': 0, 'extras': 0, 'total': 0}
{'batsman': 2, 'extras': 0, 'total': 2}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 0, 'extras': 1, 'total': 1}
{'batsman': 'SK Raina', 'bowler': 'PJ Sangwan', 'extras': {'noballs': 1}, 'non_striker': 'ML Hayden', 'runs': {'batsman': 0, 'extras': 1, 'total': 1}}
{'batsman': 0, 'extras': 0, 'total': 0}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 0, 'extras': 0, 'total': 0}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 0, 'extras': 0, 'total': 0}
{'batsman': 4, 'extras': 0, 'total': 4}
{'batsman': 2, 'extras': 0, 'total': 2}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 4, 'extras': 0, 'total': 4}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 4, 'extras': 0, 'total': 4}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 6, 'extras': 0, 'total': 6}
{'batsman': 1, 'extras': 0, 'total': 1}
{'batsman': 4, 'extras': 0, 'total': 4}
{'batsman': 0, 'extras': 1, 'total': 1}


,batsman,0,1,2,3,4,5,6,Runs,BF,NO,Team,Against,Win,Toss,team-total,season,date,match-id,SR
8,SK Raina,9,9,3,0,5,0,1,41,27,False,Chennai Super Kings,Delhi Daredevils,False,False,180,2009,2009-04-23,392189,151.292
